<span style="font-width:bold; font-size: 3rem; color:#333;">Part 01: Feature Backfill for Air Quality Data</span>

Backfill historical air quality and weather data for all sensors.



In [14]:
import datetime
import requests
import pandas as pd
import hopsworks
import json
import dotenv
import os
import great_expectations as ge

In [15]:
dotenv.load_dotenv()
project = hopsworks.login(engine="python")
fs = project.get_feature_store()
secrets = hopsworks.get_secrets_api()


2025-11-10 14:50:53,451 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-10 14:50:53,456 INFO: Initializing external client
2025-11-10 14:50:53,456 INFO: Base URL: https://c.app.hopsworks.ai:443
Connection closed.
2025-11-10 14:50:53,456 INFO: Initializing external client
2025-11-10 14:50:53,456 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-11-10 14:50:54,100 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'

2025-11-10 14:50:54,100 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'p

2025-11-10 14:50:54,905 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286307

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286307


In [16]:
AQICN_API_KEY = os.getenv("AQICN_API_KEY")
secret = secrets.get_secret("AQICN_API_KEY")
if secret is not None:
    secret.delete()
secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

In [17]:
# Sensor configuration
SENSOR_IDS = {
    "central": 1666,
    "east": 1664,
    "west": 1665,
    "north": 1662,
    "south": 1663
}

CSV_FILES = {
    1666: "../data/central,-singapore-air-quality.csv",
    1664: "../data/east,-singapore-air-quality.csv",
    1665: "../data/west,-singapore-air-quality.csv",
    1662: "../data/north,-singapore-air-quality.csv",
    1663: "../data/south,-singapore-air-quality.csv"
}

REGION_COORDS = pd.DataFrame({
    "region": ["central", "north", "south", "east", "west"],
    "latitude": [1.3521, 1.4180, 1.2800, 1.3500, 1.3400],
    "longitude": [103.8198, 103.8270, 103.8500, 103.9400, 103.7000]
})

CITY = "Singapore"
COUNTRY = "Singapore"
LATITUDE = 1.3667
LONGITUDE = 103.8



In [18]:
# Load and process air quality data for all sensors
def load_air_quality_data(sensor_id, csv_file):
    df = pd.read_csv(csv_file, parse_dates=['date'], skipinitialspace=True)
    df_aq = df[['date', 'pm25']].copy()
    df_aq['pm25'] = df_aq['pm25'].astype('float32')
    df_aq['city'] = CITY
    df_aq['sensor_id'] = sensor_id
    df_aq = df_aq[df_aq["date"] >= "2016-01-01"]

    # Count nulls
    nulls_removed = df_aq['pm25'].isna().sum()
    df_aq = df_aq.dropna()
    return df_aq, nulls_removed

all_aq_data = []
nulls_removed_list = []
for sensor_name, sensor_id in SENSOR_IDS.items():
    csv_file = CSV_FILES[sensor_id]
    df_aq, nulls_removed = load_air_quality_data(sensor_id, csv_file)
    all_aq_data.append(df_aq)
    nulls_removed_list.append(nulls_removed)
    print(f"Loaded {len(df_aq)} records for {sensor_name} (sensor_id: {sensor_id})")

# assert nulls removed are equal and less than 20
if not all(n == nulls_removed_list[0] for n in nulls_removed_list):
    raise AssertionError("Nulls removed are not equal")
if not all(n < 20 for n in nulls_removed_list):
    raise AssertionError("More than 20 nulls removed")

df_aq = pd.concat(all_aq_data, ignore_index=True)
print(f"\nTotal air quality records: {len(df_aq)}")

Loaded 3587 records for central (sensor_id: 1666)
Loaded 3586 records for east (sensor_id: 1664)
Loaded 3586 records for west (sensor_id: 1665)
Loaded 3587 records for north (sensor_id: 1662)
Loaded 3586 records for south (sensor_id: 1663)

Total air quality records: 17932


In [19]:
# Load weather data
earliest_aq_date = df_aq['date'].min().strftime('%Y-%m-%d')
url = "https://archive-api.open-meteo.com/v1/archive"

params = {
    "latitude": LATITUDE,
    "longitude": LONGITUDE,
    "start_date": earliest_aq_date,
    "format": "json",
    "end_date": datetime.datetime.now().strftime("%Y-%m-%d"),
    "daily": ["temperature_2m_mean", "precipitation_sum", "wind_speed_10m_max", "wind_direction_10m_dominant"]
}

response = requests.get(url, params=params)
if response.status_code != 200:
    raise requests.exceptions.RequestException(f"Failed to retrieve weather data. Status Code: {response.status_code}")

data = response.json()
d = data["daily"]

weather_df = pd.DataFrame({
    "date": pd.to_datetime(d["time"]),
    "temperature_2m_mean": d["temperature_2m_mean"],
    "precipitation_sum": d["precipitation_sum"],
    "wind_speed_10m_max": d["wind_speed_10m_max"],
    "wind_direction_10m_dominant": d["wind_direction_10m_dominant"],
})
weather_df["city"] = CITY

print(f"Weather records: {len(weather_df)}")

Weather records: 3602


In [20]:
# --- Feature engineering: add time-based, lag and rolling features ---
# Air quality features (per sensor)
df_aq = df_aq.sort_values(['sensor_id', 'date']).reset_index(drop=True)
# time features
df_aq['dayofweek'] = df_aq['date'].dt.dayofweek
df_aq['dayofmonth'] = df_aq['date'].dt.day
df_aq['is_weekend'] = df_aq['dayofweek'] >= 5
# lag features (1,2,3,7 days)
for lag in [1,2,3,7]:
    df_aq[f'pm_lag_{lag}'] = df_aq.groupby('sensor_id')['pm25'].shift(lag)
# rolling statistics (3-day and 7-day) - shifted by 1 to avoid leakage
for w in [3,7]:
    df_aq[f'pm_roll_mean_{w}'] = df_aq.groupby('sensor_id')['pm25'].rolling(window=w, min_periods=1).mean().shift(1).reset_index(level=0, drop=True)
    df_aq[f'pm_roll_std_{w}'] = df_aq.groupby('sensor_id')['pm25'].rolling(window=w, min_periods=1).std().shift(1).reset_index(level=0, drop=True).fillna(0.0)

# Weather features (city-level daily series): add rolling stats
weather_df = weather_df.sort_values('date').reset_index(drop=True)
for w in [3,7]:
    weather_df[f'temp_roll_mean_{w}'] = weather_df['temperature_2m_mean'].rolling(window=w, min_periods=1).mean().shift(1)
    weather_df[f'wind_speed_roll_mean_{w}'] = weather_df['wind_speed_10m_max'].rolling(window=w, min_periods=1).mean().shift(1)

# Inspect new columns
print('Air quality columns:', list(df_aq.columns))
print('Weather columns:', list(weather_df.columns))

# Optionally drop rows with NaNs introduced by lags if you plan to insert into feature store
# df_aq = df_aq.dropna().reset_index(drop=True)
# weather_df = weather_df.dropna().reset_index(drop=True)


Air quality columns: ['date', 'pm25', 'city', 'sensor_id', 'dayofweek', 'dayofmonth', 'is_weekend', 'pm_lag_1', 'pm_lag_2', 'pm_lag_3', 'pm_lag_7', 'pm_roll_mean_3', 'pm_roll_std_3', 'pm_roll_mean_7', 'pm_roll_std_7']
Weather columns: ['date', 'temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_direction_10m_dominant', 'city', 'temp_roll_mean_3', 'wind_speed_roll_mean_3', 'temp_roll_mean_7', 'wind_speed_roll_mean_7']


In [21]:
weather_df

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,temp_roll_mean_3,wind_speed_roll_mean_3,temp_roll_mean_7,wind_speed_roll_mean_7
0,2016-01-01,25.6,6.2,18.5,21,Singapore,NaN,NaN,NaN,NaN
1,2016-01-02,25.4,10.3,11.0,9,Singapore,25.600000,18.500000,25.600000,18.500000
2,2016-01-03,26.0,2.1,13.1,19,Singapore,25.500000,14.750000,25.500000,14.750000
3,2016-01-04,26.8,2.2,10.8,32,Singapore,25.666667,14.200000,25.666667,14.200000
4,2016-01-05,26.8,3.1,12.2,36,Singapore,26.066667,11.633333,25.950000,13.350000
...,...,...,...,...,...,...,...,...,...,...
3597,2025-11-06,28.1,0.1,14.2,258,Singapore,27.000000,12.633333,27.557143,10.857143
3598,2025-11-07,27.9,0.1,12.7,281,Singapore,27.500000,13.400000,27.442857,11.300000
3599,2025-11-08,28.5,0.0,14.3,288,Singapore,27.900000,12.700000,27.428571,11.485714
3600,2025-11-09,27.6,6.6,13.2,283,Singapore,28.166667,13.733333,27.471429,12.314286


In [22]:
nulls_removed = weather_df.isna().sum().sum()
nulls_removed

4

In [23]:
# Define data validation expectations
aq_expectation_suite = ge.core.ExpectationSuite(expectation_suite_name="aq_expectation_suite")
aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": True}
    )
)

weather_expectation_suite = ge.core.ExpectationSuite(expectation_suite_name="weather_expectation_suite")
for col in ["precipitation_sum", "wind_speed_10m_max"]:
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={"column": col, "min_value": -0.1, "max_value": 1000.0, "strict_min": True}
        )
    )

In [24]:
# Save sensor locations as secret
sensor_locations = []
for sensor_name, sensor_id in SENSOR_IDS.items():
    sensor_locations.append({
        "country": COUNTRY,
        "city": CITY,
        "street": sensor_name.capitalize(),
        "sensor_id": sensor_id,
        "latitude": LATITUDE,
        "longitude": LONGITUDE
    })

secret = secrets.get_secret("SENSOR_LOCATION_JSON")
if secret is not None:
    secret.delete()

secret_json = json.dumps(sensor_locations)
print(secret_json)
secrets.create_secret("SENSOR_LOCATION_JSON", secret_json)
print(f"Saved {len(sensor_locations)} sensor locations")

[{"country": "Singapore", "city": "Singapore", "street": "Central", "sensor_id": 1666, "latitude": 1.3667, "longitude": 103.8}, {"country": "Singapore", "city": "Singapore", "street": "East", "sensor_id": 1664, "latitude": 1.3667, "longitude": 103.8}, {"country": "Singapore", "city": "Singapore", "street": "West", "sensor_id": 1665, "latitude": 1.3667, "longitude": 103.8}, {"country": "Singapore", "city": "Singapore", "street": "North", "sensor_id": 1662, "latitude": 1.3667, "longitude": 103.8}, {"country": "Singapore", "city": "Singapore", "street": "South", "sensor_id": 1663, "latitude": 1.3667, "longitude": 103.8}]
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Saved 5 sensor locations
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets
Saved 5 sensor locations


In [26]:
# Create and populate air quality feature group
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city', 'sensor_id'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

air_quality_fg.insert(df_aq)

air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")
air_quality_fg.update_feature_description("sensor_id", "Sensor ID of the air quality measurement")

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1286307/fs/1265775/fg/1668627
2025-11-10 14:52:10,468 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286307/fs/1265775/fg/1668627
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286307/fs/1265775/fg/1668627


Uploading Dataframe: 100.00% |██████████| Rows 17932/17932 | Elapsed Time: 00:02 | Remaining Time: 00:00



Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286307/jobs/named/air_quality_1_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286307/jobs/named/air_quality_1_offline_fg_materialization/executions


In [27]:
# Create and populate weather feature group
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city'],
    event_time="date",
    expectation_suite=weather_expectation_suite
)

weather_fg.insert(weather_df, wait=True)

weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m above ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the day")


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1286307/fs/1265775/fg/1668628
2025-11-10 14:52:33,318 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286307/fs/1265775/fg/1668628
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286307/fs/1265775/fg/1668628


Uploading Dataframe: 100.00% |██████████| Rows 3602/3602 | Elapsed Time: 00:01 | Remaining Time: 00:00



Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286307/jobs/named/weather_1_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286307/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-10 14:52:51,141 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-10 14:52:51,141 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-10 14:52:54,324 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-10 14:52:54,324 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-10 14:55:20,654 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-11-10 14:55:20,654 INFO: Waiting for execution to finish. Current s